In [1]:
import os
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', 900)
from apyori import apriori

import queue
import threading
import time
from threading import Lock
lock = Lock()

# Loading Data

In [2]:
data_path = os.path.join(os.getcwd(), 'data')
ratings = pd.read_csv(os.path.join(data_path, 'ratings_small.csv'),  low_memory=False)
#user = pd.read_csv(os.path.join(data_path, 'users.dat'), sep='::', engine='python', names=['UserID','Gender','Age','Occupation','Zip-code'])
#movies = pd.read_csv(os.path.join(data_path, 'movies_preprocessed.csv'))

### Create a Pivot Table with 1 if the user has rated the movie and 0 if he has not done so

In [3]:
# reduced the dataset to the needed features and convert it to a matrix
user_ratings = ratings.pivot_table(index='userId', columns='movieId', values='rating').fillna(0)
user_ratings = user_ratings.astype(bool).astype(int)

### Change every 1 to the according Movie ID (Column)

In [4]:
for idx, row in user_ratings.iterrows():
    for i in user_ratings.columns:
        if row[i] != 0:
            user_ratings.at[idx, i] = i
    if(idx % 100 == 0):
        print('idx: ', idx)
        
user_ratings.head()

idx:  100
idx:  200
idx:  300
idx:  400
idx:  500
idx:  600


movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,52,53,54,55,57,58,59,60,61,62,63,64,65,66,68,69,70,71,72,73,74,76,77,78,79,80,81,82,83,84,85,86,87,88,89,92,93,94,95,96,97,98,99,100,101,102,103,104,105,107,108,110,111,112,113,114,116,117,118,119,121,122,123,124,125,126,129,130,131,132,135,137,140,141,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,183,184,185,186,187,188,189,190,191,193,194,195,196,198,199,200,201,202,203,204,205,206,207,208,209,211,213,214,215,216,217,218,219,220,222,223,224,225,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,285,287,288,289,290,291,292,293,294,295,296,299,300,301,302,303,304,305,306,307,308,309,312,313,314,315,316,317,318,319,320,321,322,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,387,388,389,390,391,392,393,401,405,407,408,409,410,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,...,113348,113378,113416,113453,113532,113565,113573,113640,113705,113741,113780,113829,113862,113938,114028,114044,114060,114074,114082,114122,114180,114265,114342,114464,114552,114601,114635,114662,114670,114707,114762,114766,114795,114818,114925,114935,115122,115147,115149,115151,115170,115174,115210,115216,115231,115502,115534,115569,115617,115624,115664,115680,115713,115877,115881,115927,116012,116136,116161,116207,116397,116413,116419,116503,116660,116797,116799,116823,116849,116855,116887,116897,116939,116977,116985,117107,117121,117123,117176,117192,117434,117444,117456,117511,117529,117533,117590,117851,117871,117895,118082,118105,118248,118260,118326,118334,118354,118468,118696,118702,118814,118880,118890,118898,118900,118924,118985,118997,119068,119141,119145,119155,119655,120392,120466,120635,120637,120783,120799,120805,120821,121113,121126,121171,121231,121491,121618,122490,122882,122886,122888,122890,122892,122900,122902,122904,122920,122924,122932,123947,124859,125916,126006,126106,126420,126430,126548,127052,127096,127098,127108,127114,127124,127136,127152,127158,127164,127178,127194,127198,127202,127204,127206,127319,127728,128360,128512,128520,128592,128606,128616,128620,128846,129009,129191,129250,129313,129354,129364,129428,129514,129653,129657,129659,129737,129937,130073,130083,130087,130351,130448,130450,130452,130490,130520,130522,130576,130580,130628,130634,130642,130682,130960,130970,130980,131013,131168,131451,131714,131724,131830,132046,132074,132146,132157,132333,132462,132480,132488,132496,132618,132796,132888,132952,132961,133195,133281,133295,133365,133377,133419,133545,133645,133771,133782,133798,133824,134025,134130,134158,134170,134246,134368,134393,134528,134569,134783,134853,134859,134881,135133,135137,135264,135266,135268,135436,135508,135518,135532,135536,135567,135569,135861,135887,136016,136018,136020,136305,136445,136447,136449,136562,136592,136598,136602,136654,136666,136800,136816,136864,137337,137403,137595,137857,138036,138204,138208,138258,138546,138696,138698,139116,139130,139385,139415,139620,139642,139644,139757,139855,139915,140110,140152,140174,140237,140247,140265,140267,140523,140711,140715,140725,140739,140741,140743,140745,140747,140749,140751,140753,140755,140757,140759,140761,140763,140816,140880,

### Creating the itemsets

In [5]:
transactions = user_ratings.values.tolist()
for i in range(0, len(transactions)):
    transactions[i] = [value for value in transactions[i]if value != 0]
    
movie_items = set()
for list_ in transactions:
    movie_items.update(list_)

movie_items = sorted(movie_items)

# Calculating Support

In [6]:
#item matrix with support of to items
def calc_support_of_column(value, column, relative=False):
    for i in range(0, len(column.index)):        
        column[column.index[i]] = calc_support(value, column.index[i], relative) 
    return column

In [7]:
def calc_support(item1, item2, relative):
    conjunction = 0
    joint = 0
    
    for transaction in transactions:
        if item1 in transaction and item2 in transaction:
            conjunction += 1
        if item1 in transaction or item2 in transaction: 
            joint += 1
    if relative:
        return conjunction/joint
    else:
        return conjunction/len(transactions)

# Creating Threads

In [8]:
exitFlag = 0
class myThread (threading.Thread):
    def __init__(self, threadID, name, q):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = name
        self.q = q
    
    def run(self):
        print ("Starting " + self.name)
        process_data(self.name, self.q)
        print ("Exiting " + self.name)

In [9]:
counter = 0
def process_data(threadName, q):
        while not exitFlag:
            queueLock.acquire()
            if not workQueue.empty():
                column_index = q.get()
                queueLock.release()
                result = calc_support_of_column(
                    support.columns[column_index], 
                    support[support.columns[column_index]].copy(), 
                    relative=True)
                with lock:
                    support[support.columns[column_index]] = result
                #if (counter % 100 == 0):
                print ("%s processing %s" % (threadName, column_index))
                counter += 1
            else:
                queueLock.release()
                time.sleep(1)

## Main

In [ ]:
support = pd.DataFrame(columns=movie_items, index=movie_items)
support[:30]
threadList = []
for i in range(1, 5):
    threadList.append(("Thread-"+str(i)))

queueLock = threading.Lock()
workQueue = queue.Queue()

threads = []
threadID = 1

# Create new threads
for tName in threadList:
    thread = myThread(threadID, tName, workQueue)
    thread.start()
    threads.append(thread)
    threadID += 1

# Fill the queue
queueLock.acquire()
print('Filling the queue...')
for i in range(0, len(support)):
    workQueue.put(i)
queueLock.release()
print('Finished filling the queue!')

# Wait for queue to empty
while not workQueue.empty():
    pass

# Notify threads it's time to exit
exitFlag = 1

# Wait for all threads to complete
for t in threads:
    t.join()
print ("Exiting Main Thread")

Starting Thread-1
Starting Thread-2
Starting Thread-3
Starting Thread-4
Filling the queue...
Finished filling the queue!


In [ ]:
support.head(100)